# Problem Statement

A US bike-sharing provider BoomBikes has recently suffered considerable dips in their revenues due to the ongoing Corona pandemic. The company is finding it very difficult to sustain in the current market scenario. So, it has decided to come up with a mindful business plan to be able to accelerate its revenue as soon as the ongoing lockdown comes to an end, and the economy restores to a healthy state. 

They have contracted a consulting company to understand the factors on which the demand for these shared bikes depends. Specifically, they want to understand the factors affecting the demand for these shared bikes in the American market. The company wants to know:

- Which variables are significant in predicting the demand for shared bikes.
- How well those variables describe the bike demands

In [1]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import packages
import pandas as pd,numpy as np
import matplotlib.pyplot as plt,seaborn as sns
import statsmodels.api as sm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Data Reading & Understanding

In [3]:
#read data
df=pd.read_csv('day.csv')


In [4]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [5]:
#view initial shape
df.shape

(730, 16)

In [6]:
#check info about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    int64  
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.4+ KB


There are no null values in the dataset

In [ ]:
#check duplicates
df.duplicated().sum()

There are no duplicate rows

In [ ]:
#statistical description
df.describe()

### Dropping irrelevant features
 - instant is index - we can drop it
 - dteday is conveying the same meaning as yr & mnth
 - casual & registered can be removed as it totals to the target value 'cnt' & possibly be a reason of data leakage

In [ ]:
df.drop(['instant','dteday','casual','registered'],axis=1,inplace=True)

In [ ]:
df.head()

## Data Preparation
#### Mapping data according to dictionary
this step is done prior to visualization so that it can have meaningful item names

In [ ]:
df.season = df.season.map({1: 'Spring',2:'Summer',3:'Fall',4:'Winter'})

In [ ]:
df.yr = df.yr.map({0: '2018',1:'2019'})

In [ ]:
import calendar
df.mnth = df.mnth.apply(lambda x:calendar.month_name[x])
df.mnth = df.mnth.apply(lambda x:x[:3])


In [ ]:
df.weekday = df.weekday.map({0:"Sun",1:"Mon",2:"Tue",3:"Wed",4:"Thr",5:"Fri",6:"Sat"})


In [ ]:
df.weathersit = df.weathersit.map({1:'Clear_FewClouds_PartlyCloudy',
                                   2:'Mist_CloudyMist_BrokenClouds_FewClouds', 
                                   3:'LightSnow_LightRain_Thunderstorm_ScatteredClouds',
                                   4:'HeavyRain_IcePallets_Thunderstorm_Mist_Snow_Fog'})


# Data Visualization

First we categorize features into categoricals & continuous

In [ ]:
cat_f=['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday',
       'weathersit']
cont_f=['temp', 'atemp', 'hum', 'windspeed','cnt']


### Univariate Analysis

In [ ]:
plt.figure(figsize=(10,10))
for i in list(enumerate(cont_f)):
    plt.subplot(3,2, i[0]+1)
    sns.boxplot(df[i[1]])
plt.tight_layout()
plt.show()

## Inferences
- There are some outliers in the hum & windspeed but it does not seem to effect much

### Bivariate Analysis

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(2,2,1)
sns.barplot(x=df.season,y=df.cnt,hue=df.yr,data=df)
plt.subplot(2,2,2)
sns.barplot(x=df.mnth,y=df.cnt,hue=df.yr,data=df)
plt.subplot(2,2,4)
sns.barplot(y=df.weathersit,x=df.cnt,hue=df.yr,data=df)
plt.show()

## Inferences
- The overall trend increases in 2019
- Summer,winter & fall sees good rentals
- the trend increases from march then dips in december
- rentals is low for LightSnow_LightRain_Thunderstorm_ScatteredClouds

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), annot = True, cmap="Greens")
plt.show()

### Inferences
- The temp & atemp is highly correlated but we will not drop them now, we will let the model decide which is more important
- hum, holiday & windspeed is negatively correlated with cnt

### Continuous Variables

Assumption 1 - Linear relationship between X & Y

The relationship is not exactly linear but temp & atemp does show some linear relationship

In [ ]:
sns.pairplot(data=df[cont_f])

### Categorical Variables

In [ ]:
plt.figure(figsize=(10,15))
for i in list(enumerate(cat_f)):
    plt.subplot(4,2, i[0]+1)
    sns.boxplot(x=df[i[1]],y=df.cnt,data=df)
    plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


## Inferences
- The number of rentals increased in 2019
- The rentals increase from summer & decreases towards winter but is still higher than Spring
- The rentals increases from Aprill but dips in July & is most in september
- There is a dip during holidays & non workingday
- There is no rentals during Heavy Rain or Ice pallets

# Data Preparation Contd..

### Dummy Variables
Creating dummy variables for categoricals

In [ ]:
dummy=df[['season','mnth','weathersit','weekday','yr']]
dummy=pd.get_dummies(dummy,drop_first=True)

df=pd.concat([df,dummy],axis=1)

In [ ]:
df.drop(['season','mnth','weathersit','weekday','yr'],axis=1,inplace=True)

In [ ]:
#present shape
df.shape

In [ ]:
df.head()

### Derived Column
Created one derived feature 'Windchill Factor' but removed it since it was left out in RFE

### Train Test Split
Splitting the dataset into train & test in 70:30 ratio

In [ ]:
np.random.seed(0)
df_train,df_test=train_test_split(df,train_size=0.7,test_size=0.3,random_state=100)

### Scaling
Scaling the training dataset

In [ ]:
scaler=MinMaxScaler()

In [ ]:
df_train[cont_f]=scaler.fit_transform(df_train[cont_f])

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

we can see the min value is 0 & max is 1 after scaling

### Dividing data into dependent & independent variables

In [ ]:
y_train=df_train.pop('cnt')
x_train=df_train


# Data Modelling and Evaluaion

First we will be building the model with all the given features in the dataset & remove them 1 by 1 using a combo of RFE & VIF/P-Val

### Model Building

In [ ]:
#Linear Regression Model
lr = LinearRegression()
lr.fit(x_train, y_train)
#dropping down features count to 15 from 30 using RFE
rfe = RFE(lr, 15)           
rfe = rfe.fit(x_train, y_train)


In [ ]:
#Checking features selected by RFE
rfe_cols=x_train.columns[rfe.support_]
rfe_cols

now using these columns we will proceed further & manually remove element as per statistical decisions following the statsmodel approach

In [ ]:
x_train=x_train[rfe_cols]

In [ ]:
#function for model creation
def model(x_train,y_train):
    x_train=sm.add_constant(x_train)
    lr=sm.OLS(y_train,x_train).fit()
    return lr,x_train

In [ ]:
#VIF function
def VIF(x_train):
    X = x_train
    features = X.columns
    Vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif=pd.DataFrame({'Features':features,'VIF':Vif})
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif

### Model 1

In [ ]:
#adding constant & fitting model
lr,x_train=model(x_train,y_train)

In [ ]:
print(lr.summary())

In [ ]:
#VIF
VIF(x_train)


Removing holiday since its P value is beyond 5%  standing at 37.3% so it is clearly insignificant & VIF is inf which means it is perfectly collinear

In [ ]:
x_train=x_train.drop(['holiday'],axis=1)

### Model 2

In [ ]:
#rebuilding without holiday
lr2,x_train=model(x_train,y_train)

In [ ]:
print(lr2.summary())

According to the P values the selected features are all significant so we will check for multicollinearity using VIF(<5 - Good)

In [ ]:
#VIF
VIF(x_train)

We see many features having VIF>5 even though the P val made it significant. We will remove the 'workingday' & check again

## Model 3

In [ ]:
x_train=x_train.drop(['workingday'],axis=1)

In [ ]:
#rebuilding without workingday
lr3,x_train=model(x_train,y_train)

In [ ]:
print(lr3.summary())

In [ ]:
#VIF
VIF(x_train)


we saw weekday_sat having a reducing VIF but the significance no longer holds true as P val is 0.218>0.05 so we remove it

## Model 4

In [ ]:
x_train=x_train.drop(['weekday_Sat'],axis=1)

In [ ]:
#rebuilding without weekday_Sat)
lr4,x_train=model(x_train,y_train)

In [ ]:
print(lr4.summary())

In [ ]:
#VIF
VIF(x_train)

season_Spring is having significance with p val 0.006 but has VIF>5 so we will drop it & rebuild model

## Model 5
Final Model


In [ ]:
x_train=x_train.drop(['season_Spring'],axis=1)

In [ ]:
#rebuilding without weekday_Sat
lr5,x_train=model(x_train,y_train)

In [ ]:
print(lr5.summary())

In [ ]:
#VIF
VIF(x_train)

we see the P val is below 5% for all the features & also there is no issue of multicollinearity present so we can finalize this model & proceed with prediction.<br>The Adj R2 obtained on training data is 0.836 we need to compare it with test data

# Residual Analysis on Training data

Here we will be checking the error terms if they are normally distributed or not

In [ ]:
y_pred_train=lr5.predict(x_train)

In [ ]:
#distribution plot of error terms
plt.suptitle("Residual Analysis on train data- Error Terms",fontsize=15)
sns.distplot((y_train-y_pred_train))
plt.xlabel("Error")
plt.show()

Assumption 2 - The error terms are normally distributed
<br>so we can proceed with it to predict on test data

## Homoscedasticity Check

In [ ]:
#distribution of error terms to check constant variance
plt.suptitle("Homscedasticity Check",fontsize=15)
a=sns.scatterplot(y_pred_train,(y_train-y_pred_train))
a=sns.lineplot([0,1],[0,0],color='red')
plt.xlabel("y_pred")
plt.ylabel("Residuals")
plt.show()

Assumption 3- Error terms have constant variance

there is no visible pattern so it can be confirmed it contains homscedascity

## Scaling
Scaling Test Data

In [ ]:
df_test[cont_f]=scaler.transform(df_test[cont_f])

In [ ]:
df_test.describe()

In [ ]:
#Separate out target feature
y_test=df_test.pop('cnt')
x_test=df_test
N=len(x_test)

In [ ]:
#use the same features in test data as on training data
x_train=x_train.drop(['const'],axis=1)
x_test=x_test[x_train.columns]

In [ ]:
#adding constant term
x_test=sm.add_constant(x_test)

In [ ]:
#predict on test data
y_pred=lr5.predict(x_test)

# Model Evaluation

In [ ]:
#R2 score on test data
r2_test=round(r2_score(y_test,y_pred),4)
print('The test data r2 score is : ', r2_test)

In [ ]:
#Adj r2 score for test data
#N= len(N)          # sample size
p =len(x_train.columns)     # Number of independent variable
r2_test_adj = round((1-((1-r2_test)*(N-1)/(N-p-1))),4)
print('Adj. R-Squared for Test dataset: ', r2_test_adj)


The model can explain 79.30% variance on test data

In [ ]:
#Actual vs Predicted
fig=plt.figure()
ax1=fig.add_subplot(111)
plt.suptitle('Actual vs Predicted',fontsize=15)
#ax1.scatter(y_test,c='b',label='y_test')
#ax1.scatter(y_pred,c='r',label='y_pred')
sns.regplot(y_test,y_pred)
plt.xlabel("y_test",fontsize=15)
plt.ylabel("y_pred",fontsize=15)
plt.show()

The actual & predicted gets overlapped which shows the model can predict the change in actual data in a good way

In [ ]:
#creating a dataframe of features & coefficients
coef=pd.DataFrame(lr5.params)
coef.insert(0,'Features',coef.index)
coef.rename(columns={0:'Coefficient'},inplace=True)
coef.reset_index(drop=True,inplace=True)
coef.sort_values(by='Coefficient',ascending=False,inplace=True)
coef

#### The top indicators of explaning the demand are 
- temp
- weathersit_LightSnow_LightRain_Thunderstorm_ScatteredClouds
- yr_2019

#### Apart from those some other indicators are
- hum
- windspeed
- season (mainly in summer & winter)
- month(sep)

BoomBikes can take above indicators into consideration & offer business benifits to attract more customers & gain profit

The Positive coefficients depicts the increase in count for those parameters & Negative coefficient signifies decrease in count




### The Equation of the best fit line obtained is

cnt = 0.598625 * temp + 0.228436 * yr_2019 + 0.135878 * season_Winter + 0.091479 * mnth_Sep + 0.082251 * season_Summer + (-0.041966) * weekday_Sun  + (-0.043909) * mnth_Jul + (-0.052904) * weathersit_Mist_CloudyMist_BrokenClouds_FewClouds + (-0.174126) * hum + (-0.189487) * windspeed + (-0.235432) * weathersit_LightSnow_LightRain_Thunderstorm_ScatteredClouds + 0.223111


## Inferences

- Temperature is a deciding factor with high coefficient
- Rentals are high during summer & winter
- Month of september sees a rise in rentals while dips in July
- Humidity & windspeed acts as a hindrance to rentals
- Weather situation for mist,snow,clouds & rain also acts negatively